In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
!git clone -q https://github.com/coleman-zachery/poker_pytorch.git git_download
!rm -rf libs
!mv git_download/libs .
!rm -rf git_download

from libs.constants import RANKS, SUITS
from libs.helpers import card_strings_to_tensor, card_str_to_tuple, classify_poker_hand, poker_hand_label_to_index, reverse_poker_hand_index
from libs.encoders import CardEncoder, PokerHandClassifier
from libs.generators import generate_straight_flush, generate_four_of_a_kind

In [ ]:
generate_four_of_a_kind()

In [ ]:
generate_straight_flush()

In [6]:
# ---- training set ----
card_strings = [
    ["A♥", "K♦", "Q♣", "J♠", "__"], # high card
    ["A♥", "K♥", "Q♥", "J♥", "X♥"], # royal flush
    ["9♠", "8♠", "7♠", "6♠", "5♠"], # straight flush
    ["3♦", "3♣", "3♥", "3♠", "9♥"], # four of a kind
    ["4♥", "4♠", "4♣", "9♦", "9♥"], # full house
    ["2♣", "5♣", "9♣", "J♣", "K♣"], # flush
    ["6♥", "5♠", "4♦", "3♣", "2♥"], # straight
    ["7♠", "7♥", "7♣", "2♦", "5♥"], # three of a kind
    ["8♦", "8♣", "4♥", "4♠", "K♥"], # two pair
    ["J♣", "J♠", "3♥", "6♦", "9♣"], # one pair
    ["5♥", "X♠", "7♣", "4♦", "2♥"], # high card
    ["5♥", "5♠", "__", "__", "__"], # one pair with 3 empty cards
    ["9♣", "9♦", "4♥", "4♠", "??"], # two pair and 1 ?? card
    ["__", "__", "__", "__", "__"], # empty hand
]


In [7]:
# create tensors and labels
card_tensors = []
labels = []
for hand in card_strings:
    card_tensor = card_strings_to_tensor(hand)
    card_tensors.append(card_tensor)
    card_tuples = [card_str_to_tuple(cs) for cs in hand]
    label = classify_poker_hand(card_tuples)
    labels.append(poker_hand_label_to_index(label))

# get classification embeddings
card_tensors = torch.stack(card_tensors)
labels = torch.tensor(labels, dtype=torch.long)
# print("Card Tensors:\n", card_tensors)
# print("Labels:\n", labels)
encoder = CardEncoder()
classifier = PokerHandClassifier() # ---- choose classifier ----
encoded_cards = encoder(card_tensors.view(-1, 2))
encoded_hands = encoded_cards.view(card_tensors.size(0), 5, -1)
outputs = classifier(encoded_hands)
print("Classifier Outputs:\n", outputs)

Classifier Outputs:
 tensor([[-0.0901,  0.0912,  0.1677, -0.3872,  0.1904,  0.1035,  0.0020,  0.2013,
          0.2628,  0.0254,  0.3606],
        [-0.1770,  0.1669,  0.1383, -0.3548,  0.1152,  0.0228,  0.1605,  0.3850,
          0.4458,  0.2751,  0.2645],
        [-0.1252,  0.0081,  0.1945, -0.2123,  0.0614, -0.0961, -0.1242,  0.0680,
          0.2042,  0.0365,  0.1292],
        [-0.1310,  0.0152,  0.2057, -0.2429, -0.0096, -0.1942, -0.0437,  0.1100,
          0.2461,  0.1112,  0.2917],
        [-0.0173,  0.0893, -0.0197, -0.3045,  0.1167, -0.0467,  0.0646,  0.2514,
          0.2999,  0.1121,  0.2813],
        [-0.0066,  0.0282,  0.1498, -0.2300,  0.1696,  0.0391, -0.1267,  0.1427,
          0.2548,  0.0143,  0.2427],
        [-0.0784,  0.0794,  0.0813, -0.3288,  0.0493, -0.0754, -0.0523,  0.2080,
          0.2154,  0.0199,  0.0983],
        [-0.1149, -0.0433,  0.0888, -0.3684,  0.0528, -0.0871,  0.0511,  0.3364,
          0.1926,  0.0613,  0.1576],
        [-0.1475,  0.1256,  0.1675,

In [8]:
# ---- train loop ----
def train_loop(model, data_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for batch_cards, batch_labels in data_loader:
            optimizer.zero_grad()
            encoded_cards = encoder(batch_cards.view(-1, 2))
            encoded_hands = encoded_cards.view(batch_cards.size(0), 5, -1)
            outputs = model(encoded_hands)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(data_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Example of setting up a data loader and training
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(card_tensors, labels)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
train_loop(classifier, data_loader, criterion, optimizer, num_epochs=100)


Epoch [1/100], Loss: 2.4596
Epoch [2/100], Loss: 2.3776
Epoch [3/100], Loss: 2.3298
Epoch [4/100], Loss: 2.3025
Epoch [5/100], Loss: 2.2648
Epoch [6/100], Loss: 2.2086
Epoch [7/100], Loss: 2.2008
Epoch [8/100], Loss: 2.1844
Epoch [9/100], Loss: 2.0987
Epoch [10/100], Loss: 2.0614
Epoch [11/100], Loss: 2.0142
Epoch [12/100], Loss: 1.9918
Epoch [13/100], Loss: 1.9270
Epoch [14/100], Loss: 1.8825
Epoch [15/100], Loss: 1.8313
Epoch [16/100], Loss: 1.7853
Epoch [17/100], Loss: 1.7226
Epoch [18/100], Loss: 1.6959
Epoch [19/100], Loss: 1.6075
Epoch [20/100], Loss: 1.5359
Epoch [21/100], Loss: 1.5689
Epoch [22/100], Loss: 1.4546
Epoch [23/100], Loss: 1.4088
Epoch [24/100], Loss: 1.3952
Epoch [25/100], Loss: 1.3499
Epoch [26/100], Loss: 1.2350
Epoch [27/100], Loss: 1.1888
Epoch [28/100], Loss: 1.0729
Epoch [29/100], Loss: 1.1546
Epoch [30/100], Loss: 1.0738
Epoch [31/100], Loss: 1.0008
Epoch [32/100], Loss: 0.9919
Epoch [33/100], Loss: 0.9081
Epoch [34/100], Loss: 0.8560
Epoch [35/100], Loss: 0

In [9]:
# ---- function that outputs the softmax probability for each class >= 1.00%, formatted to .00% and sorted largest to smallest % ----
def get_class_probabilities(model, hand_tensor):
    model.eval()
    with torch.no_grad():
        encoded_cards = encoder(hand_tensor.view(-1, 2))
        encoded_hand = encoded_cards.view(1, 5, -1)
        outputs = model(encoded_hand)
        probabilities = F.softmax(outputs, dim=1).squeeze(0)
        significant_probs = {reverse_poker_hand_index(i): prob.item() for i, prob in enumerate(probabilities)}
        sorted_probs = dict(sorted(significant_probs.items(), key=lambda item: item[1], reverse=True))
        formatted_probs = {k: f"{v*100:.2f}%" for k, v in sorted_probs.items()}
        return formatted_probs

In [10]:
# ---- print tensor hands, expected output, and classifier outputs ----
for i in range(card_tensors.size(0)):
    hand_tensor = card_tensors[i].unsqueeze(0)
    expected_label = labels[i].item()
    hand_type = reverse_poker_hand_index(expected_label)
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    #predicted_label = torch.argmax(outputs, dim=1).item()
    hand = card_strings[i]
    print(f"\nhand: {hand}")
    print(f"correct label confidence: {hand_type} -- {probabilities[hand_type]}")

# ---- test to see if the model can correctly predict each royal flush hand ----
royal_flush_hands = [
    ["A♣", "K♣", "Q♣", "J♣", "X♣"],
    ["A♦", "K♦", "Q♦", "J♦", "X♦"],
    ["A♥", "K♥", "Q♥", "J♥", "X♥"],
    ["A♠", "K♠", "Q♠", "J♠", "X♠"],
]

for hand in royal_flush_hands:
    hand_tensor = card_strings_to_tensor(hand).unsqueeze(0)
    hand_type = "royal flush"
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    print(f"\nhand: {hand}")
    print(f"correct label confidence: {hand_type} -- {probabilities[hand_type]}")



hand: ['A♥', 'K♦', 'Q♣', 'J♠', '__']
correct label confidence: high card -- 95.68%

hand: ['A♥', 'K♥', 'Q♥', 'J♥', 'X♥']
correct label confidence: straight flush -- 96.39%

hand: ['9♠', '8♠', '7♠', '6♠', '5♠']
correct label confidence: straight flush -- 94.81%

hand: ['3♦', '3♣', '3♥', '3♠', '9♥']
correct label confidence: four of a kind -- 95.56%

hand: ['4♥', '4♠', '4♣', '9♦', '9♥']
correct label confidence: full house -- 93.42%

hand: ['2♣', '5♣', '9♣', 'J♣', 'K♣']
correct label confidence: flush -- 91.82%

hand: ['6♥', '5♠', '4♦', '3♣', '2♥']
correct label confidence: straight -- 92.90%

hand: ['7♠', '7♥', '7♣', '2♦', '5♥']
correct label confidence: three of a kind -- 95.21%

hand: ['8♦', '8♣', '4♥', '4♠', 'K♥']
correct label confidence: two pair -- 98.51%

hand: ['J♣', 'J♠', '3♥', '6♦', '9♣']
correct label confidence: one pair -- 96.38%

hand: ['5♥', 'X♠', '7♣', '4♦', '2♥']
correct label confidence: high card -- 92.27%

hand: ['5♥', '5♠', '__', '__', '__']
correct label confidenc

In [ ]:
royal_flush_hands = [
    ["__", "A♣", "K♦", "Q♥", "J?"],
]

for hand in royal_flush_hands:
    hand_tensor = card_strings_to_tensor(hand).unsqueeze(0)
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    print(f"\nhand: {hand}")
    print(f"class probabilities: {probabilities}")